## Log Gaussian Cox process
This notebook implements the log gaussian cox process in dimension 64^2

In [ ]:
import numpy as np
import numexpr as ne
from numba import jit
from matplotlib import pyplot as plt
import sys
sys.path.append("/home/alex/Dropbox/smc_hmc/python_smchmc/")
#sys.path.append("/home/alex/Dropbox/smc_hmc/python_smchmc/smc_sampler_functions")

Define the covariance functions

In [ ]:
dim = 400
from smc_sampler_functions.target_distributions_logcox import f_dict_log_cox
parameters_log_cox = f_dict_log_cox(int(dim**0.5))


In [ ]:
N_particles = 2**9
T_time = 40
move_steps_hmc = 0
move_steps_rw_mala = 0
ESStarget = 0.5
M_num_repetions = 1
epsilon = 1.
epsilon_hmc = .1
verbose = False
parameters = {'dim' : dim, 
              'N_particles' : N_particles, 
              'T_time' : T_time,
              'autotempering' : False,
              'ESStarget': ESStarget,
              'adaptive_covariance' : True
             }
from smc_sampler_functions.functions_smc_help import sequence_distributions
from smc_sampler_functions.proposal_kernels import proposalmala, proposalrw, proposalhmc, proposalhmc_parallel
from smc_sampler_functions.functions_smc_main import smc_sampler
maladict = {'proposalkernel_tune': proposalmala,
                      'proposalkernel_sample': proposalmala,
                      'proposalname' : 'MALA',
                      'target_probability' : 0.65,
                      'covariance_matrix' : np.eye(dim), 
                      'L_steps' : 1,
                      'epsilon' : np.array([epsilon]),
                      'epsilon_max' : np.array([epsilon]),
                      'tune_kernel': True,
                      'sample_eps_L' : True,
                      'verbose' : verbose,
                      'move_steps': move_steps_rw_mala
                      }
hmcdict = {'proposalkernel_tune': proposalhmc,
                      'proposalkernel_sample': proposalhmc_parallel,
                      'proposalname' : 'HMC',
                      'target_probability' : 0.9,
                      'covariance_matrix' : np.eye(dim), 
                      'L_steps' : 20,
                      'epsilon' : np.array([epsilon_hmc]),
                      'epsilon_max' : np.array([epsilon_hmc]),
                      'accept_reject' : True,
                      'tune_kernel': True,
                      'sample_eps_L' : True,
                      'parallelize' : False,
                      'verbose' : verbose,
                      'move_steps': move_steps_hmc,
                      'mean_L' : False
                      }

from smc_sampler_functions.functions_smc_main import repeat_sampling
from smc_sampler_functions.target_distributions_logcox import priorlogdens_log_cox, priorgradlogdens_log_cox, priorsampler_log_cox
from smc_sampler_functions.target_distributions_logcox import targetlogdens_log_cox, targetgradlogdens_log_cox


parameters.update(parameters_log_cox)

priordistribution = {'logdensity' : priorlogdens_log_cox, 'gradlogdensity' : priorgradlogdens_log_cox, 'priorsampler': priorsampler_log_cox}
targetdistribution = {'logdensity' : targetlogdens_log_cox, 'gradlogdensity' : targetgradlogdens_log_cox, 'target_name': 'log_cox'}
samplers_list_dict = [maladict, hmcdict]


temperedist = sequence_distributions(parameters, priordistribution, targetdistribution)
res_repeated_sampling, res_first_iteration = repeat_sampling(samplers_list_dict, temperedist,  parameters, M_num_repetions=1, save_res=False, save_name = '')






In [ ]:
from smc_sampler_functions.functions_smc_plotting import plot_repeated_simulations, plot_results_single_simulation
plot_repeated_simulations(res_repeated_sampling)
plot_results_single_simulation(res_first_iteration)


In [ ]:
x_res_matrix1 = res_first_iteration[0]['particles_resampled'].mean(axis=0).reshape(parameters_log_cox['N'],parameters_log_cox['N'])
x_res_matrix2 = res_first_iteration[1]['particles_resampled'].mean(axis=0).reshape(parameters_log_cox['N'],parameters_log_cox['N'])

Y_matrix = parameters_log_cox['Y'].reshape(parameters_log_cox['N'],parameters_log_cox['N'])
X_matrix = parameters_log_cox['X_true'].reshape(parameters_log_cox['N'],parameters_log_cox['N'])

plt.figure(figsize=(16,16))
plt.subplot(141)
plt.title('Latent process')
im = plt.imshow(X_matrix,cmap='jet')
plt.colorbar(im,fraction=0.046, pad=0.04)

plt.subplot(142)
plt.title('Recovered latent process mala')
im = plt.imshow(x_res_matrix1,cmap='jet')
plt.colorbar(im,fraction=0.046, pad=0.04)

plt.subplot(143)
plt.title('Recovered latent process hmc')
im = plt.imshow(x_res_matrix2, cmap='jet')
plt.colorbar(im,fraction=0.046, pad=0.04)


plt.subplot(144)
plt.title('Observed Data')
im = plt.imshow(Y_matrix,cmap='jet')
plt.colorbar(im,fraction=0.046, pad=0.04)
plt.show()
